In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [3]:
PROPHET_PATH = '../data/prophet/'

In [4]:
glob.glob(PROPHET_PATH+'*')

['../data/prophet/ds.f',
 '../data/prophet/page_index.f',
 '../data/prophet/pagedf.f',
 '../data/prophet/v0']

In [5]:
pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')
ds = pd.read_feather(PROPHET_PATH+'ds.f')

## Version X
Should set version directory name in next cell. Should describe version specifics (outliers, holidays, validation period). How will the validation period be communcated to the SMAPE scoring stuff?

## Val indexing on -60
## No outlier

In [7]:
# should break if the dir already exists - avoids accidental overwriting
VERSION = 'v0/'
val_lims = (0,-60)
os.makedirs(PROPHET_PATH+VERSION)

FileExistsError: [Errno 17] File exists: '../data/prophet/v0/'

In [8]:
pagedf = pagedf.loc[:,:'10']

In [19]:
def process_page(page):
    df = ds.join(pagedf[page])
    df.columns = ['ds','y']
    # note this is doing validation on last 60 days
    # should also consider doing validation on the time period we are forcasting
    traindf = df.iloc[val_lims[0]:val_lims[1]].copy()
    traindf['train'] = 1 # feather won't serialize bool so 1s and 0s...
    # do outlier removal here
    #traindf.loc[traindf.y > traindf.y.quantile(.95), ['y']] = None
    m = Prophet(yearly_seasonality=True)
    m.fit(traindf)
    forecast = m.predict(ds)
    forecast = forecast.join(df.y)
    forecast = forecast.join(traindf.loc[:,['train']])
    forecast['train'][forecast['train'] != 1] = 0 # 0 bools
    forecast.to_feather(PROPHET_PATH+VERSION+page+'df.f')
    with open(PROPHET_PATH+VERSION+page+'m.pk', 'wb') as file:
        pk.dump(m,file)

In [20]:
mp.cpu_count()

4

In [21]:
with utils.clock():
    mp_pool = mp.Pool()
    with tqdm(total=pagedf.shape[1]) as pbar:
        for i, _ in tqdm(enumerate(mp_pool.imap_unordered(process_page, pagedf))):
            #clear_output() # added to avoid the buggy multiple lines of tqdm output - open to improvements...
            pbar.update()           
    pbar.close()
    mp_pool.close()
    mp_pool.join()


  0%|          | 0/11 [00:00<?, ?it/s]

0it [00:00, ?it/s]/Users/dbcerigo/miniconda3/envs/basePy3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app

  9%|▉         | 1/11 [00:02<00:26,  2.68s/it]

1it [00:02,  2.68s/it]/Users/dbcerigo/miniconda3/envs/basePy3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/dbcerigo/miniconda3/envs/basePy3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

Elapsed time 7.437420129776001 seconds


In [22]:
pd.read_feather(PROPHET_PATH+VERSION+'4df.f')

,ds,t,trend,seasonal_lower,seasonal_upper,trend_lower,trend_upper,yhat_lower,yhat_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,seasonal,yhat,y,train
0,2015-07-01,-1.469697,71.642638,-10.574879,-10.574879,71.642638,71.642638,33.967433,88.312768,4.833918,4.833918,4.833918,-15.408796,-15.408796,-15.408796,-10.574879,61.067759,NaN,1.0
1,2015-07-02,-1.464646,71.493785,-19.628123,-19.628123,71.493785,71.493785,22.508030,79.642885,-4.525515,-4.525515,-4.525515,-15.102607,-15.102607,-15.102607,-19.628123,51.865663,NaN,1.0
2,2015-07-03,-1.459596,71.344932,-16.423739,-16.423739,71.344932,71.344932,27.204445,82.683066,-1.656370,-1.656370,-1.656370,-14.767370,-14.767370,-14.767370,-16.423739,54.921193,NaN,1.0
3,2015-07-04,-1.454545,71.196080,-8.626254,-8.626254,71.196080,71.196080,34.922128,92.043894,5.782635,5.782635,5.782635,-14.408889,-14.408889,-14.408889,-8.626254,62.569826,NaN,1.0
4,2015-07-05,-1.449495,71.047227,-15.015912,-15.015912,71.047227,71.047227,26.603096,82.216312,-0.982546,-0.982546,-0.982546,-14.033366,-14.033366,-14.033366,-15.015912,56.031315,NaN,1.0
5,2015-07-06,-1.444444,70.898375,-12.364108,-12.364108,70.898375,70.898375,29.317237,87.377462,1.283094,1.283094,1.283094,-13.647201,-13.647201,-13.647201,-12.364108,58.534267,NaN,1.0
6,2015-07-07,-1.439394,70.749522,-17.992012,-17.992012,70.749522,70.749522,24.340770,80.807966,-4.735216,-4.735216,-4.735216,-13.256797,-13.256797,-13.256797,-17.992012,52.757510,NaN,1.0
7,2015-07-08,-1.434343,70.600669,-8.034448,-8.034448,70.600669,70.600669,33.306851,91.106988,4.833918,4.833918,4.833918,-12.868366,-12.868366,-12.868366,-8.034448,62.566222,NaN,1.0
8,2015-07-09,-1.429293,70.451817,-17.013271,-17.013271,70.451817,70.451817,23.471671,80.007232,-4.525515,-4.525515,-4.525515,-12.487755,-12.487755,-12.487755,-17.013271,53.438546,NaN,1.0
9,2015-07-10,-1.424242,70.302964,-13.776657,-13.776657,70.302964,70.302964,27.370316,84.800561,-1.656370,-1.656370,-1.656370,-12.120287,-12.120287,-12.120287,-13.776657,56.526307,NaN,1.0
